## Representation Testing

In [1]:
import numpy as np

# Import Qiskit
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram, plot_state_city
import qiskit.quantum_info as qi
import re

simulator = AerSimulator()

Representation

In [12]:
# Parameters
qubits = 3
max_circuit_depth = 5

# Initialize circuit
test_circuit = QuantumCircuit(qubits)

# Gate map
chromosome_qiskit_gate_map = {
    "w": test_circuit.barrier,  # Use barrier for blank "wires"
    "-": None,  # Used for control qubits
    "x": test_circuit.x,
    "y": test_circuit.y,
    "z": test_circuit.z,
    "h": test_circuit.h,
    "s": test_circuit.s,
    "sdg": test_circuit.sdg,
    "t": test_circuit.t,
    "cx": test_circuit.cx,
    "cy": test_circuit.cy,
    "cz": test_circuit.cz,
    "ch": test_circuit.ch,
    "cs": test_circuit.cs,
    "swap": test_circuit.swap,
}

# Example chromosome
test_circuit_chromosome = [
    ['x', 'x', 'x'],
    ['h', 'w', 'w'],
    ['cs(1,0)', '-', 'w'],
    ['cz(2,0)', 'w', '-'],
    ['sdg', 'w', 'w'],
    ['t', 'w', 'w'],
    ['cz(2,0)', 'w', '-'],
    ['s', 'w', 'w'],
    ['swap(2,0)', 'w', '-'],
    ['w', 'h', 'w'],
    ['w', 'cs(2,1)', '-'],
    ['w', 'w', 'h'],
    ['swap(2,0)', 'w', '-'],
]

# Helper to apply gates
for block in test_circuit_chromosome:
    for qubit in range(len(block)):
        gate_spec = block[qubit]
        if gate_spec == "-":
            continue
        elif "(" in gate_spec:
            gate, args = re.match(r"(\w+)\((.+)\)", gate_spec).groups()
            args = list(map(int, args.split(",")))
            chromosome_qiskit_gate_map[gate](*args)
        else:
            chromosome_qiskit_gate_map[gate_spec](qubit)

# Add save statevector
test_circuit.save_statevector()

# Output
print(test_circuit)
print("Circuit depth:", test_circuit.depth())


        ┌───┐┌───┐┌───┐   ┌─────┐┌───┐      ┌───┐       ░   ░    ░             »
   q_0: ┤ X ├┤ H ├┤ S ├─■─┤ Sdg ├┤ T ├────■─┤ S ├────X──░───░────░────────X────»
        ├───┤└─░─┘└─┬─┘ │ └──░──┘└─░─┘ ░  │ └─░─┘ ░  │  ░ ┌───┐┌───┐  ░   │  ░ »
   q_1: ┤ X ├──░────■───┼────░─────░───░──┼───░───░──┼──░─┤ H ├┤ S ├──░───┼──░─»
        ├───┤  ░    ░   │    ░     ░   ░  │   ░   ░  │  ░ └───┘└─┬─┘┌───┐ │  ░ »
   q_2: ┤ X ├──░────░───■────░─────░──────■───░──────X──░────────■──┤ H ├─X────»
        └───┘  ░    ░        ░     ░          ░         ░           └───┘      »
meas: 3/═══════════════════════════════════════════════════════════════════════»
                                                                               »
«         ░ ┌─┐      
«   q_0: ─░─┤M├──────
«         ░ └╥┘┌─┐   
«   q_1: ─░──╫─┤M├───
«         ░  ║ └╥┘┌─┐
«   q_2: ─░──╫──╫─┤M├
«         ░  ║  ║ └╥┘
«meas: 3/════╩══╩══╩═
«            0  1  2 
Circuit depth: 13


Running Circuit

In [23]:
# Transpile for simulator
simulator = AerSimulator(method='statevector')
transpiled_test_circuit = transpile(test_circuit, simulator)

# Run and get statevector
result = simulator.run(transpiled_test_circuit).result()
statevector = result.get_statevector(transpiled_test_circuit)
plot_state_city(statevector, title='Bell state')

QiskitError: 'No statevector for experiment "<qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x0000023346CAE8D0>"'